In [ ]:
params_file = 'models_4/FPN_224_f3_efc38c'
model_no = '.22'
eval_crop_size = 224

In [ ]:
from run_seg_test import LoadModelParams
params = LoadModelParams(params_file)
params.load_model_from = params_file + model_no + '.model'

# Load data

In [ ]:
import sys
#sys.path.append(r'D:\Programming\3rd_party\keras')

In [ ]:
import sys
from imp import reload
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
import pandas as pd

import keras

from keras.models import Model, load_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

import tensorflow as tf

from skimage.transform import resize
import cv2
from tqdm import tqdm_notebook

In [ ]:
import load_data
load_data = reload(load_data)

In [ ]:
DEV_MODE_RANGE = 0 # off

In [ ]:
train_df = load_data.LoadData(train_data = True, DEV_MODE_RANGE = DEV_MODE_RANGE)

In [ ]:
test_df = load_data.LoadData(train_data = False, DEV_MODE_RANGE = DEV_MODE_RANGE)

In [ ]:
test_fold_no = params.test_fold_no
test_fold_no

In [ ]:
train_images, train_masks, validate_images, validate_masks = load_data.SplitTrainData(train_df, test_fold_no)
validate_images.shape, validate_masks.shape

In [ ]:
test_images = test_df.images
test_images.shape

# IOU metric

In [ ]:
thresholds = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])

def iou(img_true, img_pred):
    assert (img_true.shape[-1]==1) and (len(img_true.shape)==3) or (img_true.shape[-1]!=1) and (len(img_true.shape)==2)
    i = np.sum((img_true*img_pred) >0)
    u = np.sum((img_true + img_pred) >0)
    if u == 0:
        return 1
    return i/u

def iou_metric(img_true, img_pred):
    img_pred = img_pred > 0.5 # added by sgx 20180728
    if img_true.sum() == img_pred.sum() == 0:
        scores = 1
    else:
        scores = (thresholds <= iou(img_true, img_pred)).mean()
    return scores

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = len(y_true_in)
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    #print("metric = ",metric)
    return np.mean(metric)

# adapter for Keras
def my_iou_metric(label, pred):
    metric_value = tf.py_func(iou_metric_batch, [label, pred], tf.float64)
    return metric_value

# model

In [ ]:

#del AlbuDataGenerator

In [ ]:
sys.path.append('../3rd_party/segmentation_models')
import segmentation_models
segmentation_models = reload(segmentation_models)
from segmentation_models.utils import set_trainable
from my_augs import AlbuDataGenerator

In [ ]:
from distutils.version import StrictVersion

In [ ]:
StrictVersion(keras.__version__)

In [ ]:
if StrictVersion(keras.__version__) < StrictVersion('2.2.3'):
    print('Old Keras {}'.format(StrictVersion(keras.__version__)))
    UpSampling2DLayerClass = segmentation_models.fpn.layers.UpSampling2D
else:
    UpSampling2DLayerClass = keras.layers.UpSampling2D
custom_objects={'my_iou_metric': my_iou_metric,
                'UpSampling2D': UpSampling2DLayerClass}
print(UpSampling2DLayerClass)


In [ ]:
model1 = load_model(params.load_model_from,
                   #custom_objects={'my_iou_metric': my_iou_metric} + custom_objects
                   custom_objects=custom_objects
                   )
print('MODEL LOADED from: ' + params.load_model_from)

In [ ]:
model = None
if 'interpolation' in params.model_params and params.model_params['interpolation']=='bilinear':
    print('Rebuilding model to fix BILINEAR problem')
    if params.model == 'FNN':
        model = segmentation_models.FPN(backbone_name=params.backbone, input_shape=(None, None, params.channels),
                                        encoder_weights=params.initial_weightns, freeze_encoder=True,
                                        dropout = params.dropout,
                                                **params.model_params)
    if params.model == 'FNNdrop':
        model = segmentation_models.FPNdrop(backbone_name=params.backbone, input_shape=(None, None, params.channels),
                                        encoder_weights=params.initial_weightns, freeze_encoder=True,
                                        dropout = params.dropout,
                                                **params.model_params)
    if params.model == 'Unet':
        model = segmentation_models.Unet(backbone_name=params.backbone, input_shape=(None, None, params.channels),
                                         encoder_weights=params.initial_weightns, freeze_encoder=True,
                                                **params.model_params)
    if params.model == 'Linknet':
        model = segmentation_models.Linknet(backbone_name=params.backbone, input_shape=(None, None, params.channels),
                                            encoder_weights=params.initial_weightns, freeze_encoder=True,
                                                **params.model_params)
    if params.model == 'divrikwicky':
        model = keras_unet_divrikwicky_model.CreateModel(params.nn_image_size,
                                                **params.model_params)
        params.backbone = ''
    model.set_weights(model1.get_weights())
else:
    model = model1
assert model

In [ ]:
optimizer=params.optimizer
if optimizer == 'adam':
    optimizer = keras.optimizers.adam(**params.optimizer_params)
elif optimizer == 'sgd':
    optimizer = keras.optimizers.sgd(**params.optimizer_params)


In [ ]:
for l in model.layers:
    if isinstance(l, UpSampling2DLayerClass):
        if hasattr(l, 'interpolation'):
            print(l.interpolation)
            #l.interpolation = params.interpolation
        else:
            print('qq')
        

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["acc", my_iou_metric]) #, my_iou_metric


# Test (try to reproduce) train validaton

In [ ]:
mean_val = 0.481577
mean_std = 0.11108
mean_val, mean_std 


In [ ]:
val_gen = AlbuDataGenerator(validate_images, validate_masks, batch_size=params.test_batch_size,nn_image_size = params.nn_image_size,
                            mode = 'inference', shuffle=False, params = params, mean=(mean_val, mean_std),
                           use_ceil = False)


In [ ]:
model.evaluate_generator(val_gen, steps=1*len(val_gen), max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

# Train evaluation

In [ ]:
sys.path.insert(1, '../3rd_party/albumentations')
sys.path.insert(1, '../3rd_party/imgaug')
import albumentations

In [ ]:
mean_val = 0.481577
mean_std = 0.11108
mean_val, mean_std 

In [ ]:
orig_nn_image_size = params.nn_image_size
params.nn_image_size = eval_crop_size #params.padded_image_size

In [ ]:
from my_augs import AlbuDataGenerator


In [ ]:
val_gen = AlbuDataGenerator(validate_images, validate_masks, batch_size=params.test_batch_size, nn_image_size = params.nn_image_size,
                            mode = 'inference', shuffle=False, params = params, mean=(mean_val, mean_std),
                           use_ceil = False)

In [ ]:
sys.path.append('../3rd_party/keras-tqdm')
from keras_tqdm import TQDMCallback, TQDMNotebookCallback

In [ ]:
r = model.predict_generator(val_gen, max_queue_size=10, workers=1, use_multiprocessing=False)

In [ ]:
r.shape

In [ ]:
r = r[:validate_images.shape[0], ...] # if ceil, r dim is higher

In [ ]:
type(r), r.shape

In [ ]:
start_coord = (params.nn_image_size - params.augmented_image_size)//2
start_coord, r.shape 

In [ ]:
r_orig = r[:, start_coord : start_coord + params.augmented_image_size, start_coord : start_coord + params.augmented_image_size]

In [ ]:
type(r_orig), r_orig.shape

In [ ]:
r_orig[0, :,:,0].shape

In [ ]:
valid_results = []
for i in range(r.shape[0]):
    valid_results += [cv2.resize(r_orig[i, :,:,0], (101,101))]

In [ ]:
validate_masks[0].shape, valid_results[0].shape

In [ ]:
validate_masks = validate_masks[:len(valid_results)]

In [ ]:
m = iou_metric_batch(validate_masks, valid_results)
print(m)

# Optimal threshold

In [ ]:
## Scoring for last model
thresholds_list = np.linspace(0.3, 0.7, 31)
ious = np.array([iou_metric_batch(validate_masks, [np.array(img,dtype = np.float32) for img in np.asarray(valid_results) > threshold]) for threshold in tqdm_notebook(thresholds_list)])


In [ ]:
threshold_best_index = np.argmax(ious) 
iou_best = ious[threshold_best_index]
threshold_best = thresholds_list[threshold_best_index]

plt.plot(thresholds_list, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

In [ ]:
threshold_best

# Single model output

In [ ]:
test_images.shape, test_images[0].shape

In [ ]:
test_gen = AlbuDataGenerator(test_images, None, batch_size=params.test_batch_size, nn_image_size = params.nn_image_size,
                            mode = 'inference', shuffle=False, params = params, mean=(mean_val, mean_std),
                           use_ceil = True)

In [ ]:
r = model.predict_generator(test_gen, max_queue_size=10, workers=1, use_multiprocessing=False)

In [ ]:
r.shape

In [ ]:
r = r[:test_images.shape[0], ...] # if ceil, r dim is higher

In [ ]:
type(r), r.shape

In [ ]:
start_coord = (params.nn_image_size - params.augmented_image_size)//2
start_coord, r.shape 

In [ ]:
r_orig = r[:, start_coord : start_coord + params.augmented_image_size, start_coord : start_coord + params.augmented_image_size]

In [ ]:
type(r_orig), r_orig.shape

In [ ]:
r_orig[0, :,:,0].shape

In [ ]:
test_results = []
for i in range(r.shape[0]):
    test_results += [cv2.resize(r_orig[i, :,:,0], (101,101))]

In [ ]:
for i,im in enumerate(test_images):
    if np.sum(im) == 0:
        test_results[i][...] = 0
        

In [ ]:
len(test_results), test_results[0].shape

In [ ]:
np.save(params_file + model_no + '.results', test_results)

In [ ]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
import time
t1 = time.time()
pred_dict = {idx: rle_encode((test_results[i] > threshold_best)) for i, idx in enumerate(tqdm_notebook(test_df.index.values))}
t2 = time.time()

print(f"Usedtime = {t2-t1} s")

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv(params_file + model_no+'.submission.csv.gz', compression = 'gzip')

In [ ]:
sub.head(10)